# 예제 4.16-4.24: 이미지 데이터 증강 (Image Data Augmentation)

## 학습목표
1. **이미지 데이터 증강** 개념 이해하기
2. **torchvision.transforms** 사용법 익히기
3. **다양한 이미지 증강 기법** 학습하기
   - 크기 조정 (Resize)
   - 회전, 뒤집기 (Rotation, Flip)
   - 자르기, 패딩 (Crop, Pad)
   - 아핀 변환 (Affine)
   - 색상 변환 (ColorJitter)
   - 노이즈, 날씨 효과 (imgaug)
   - 랜덤 삭제 (Random Erasing)
   - Mixup

---

#### 예제 4.16 라이브러리 설치

In [ ]:
# imgaug: 고급 이미지 증강 라이브러리
!pip install imgaug

---

#### 시각화를 위한 matplotlib 임포트

In [ ]:
# 이미지 시각화용 matplotlib
from matplotlib import pyplot as plt

---

#### 예제 4.17 기본 변환 (Resize, ToTensor)

이미지 크기 조정 및 텐서 변환

In [ ]:
from PIL import Image
from torchvision import transforms


# transforms.Compose: 여러 변환을 순차적으로 적용
transform = transforms.Compose(
    [
        transforms.Resize(size=(512, 512)),  # 이미지 크기 조정
        transforms.ToTensor()  # PIL 이미지 → 텐서 변환 (0~1 정규화)
    ]
)

# 이미지 로드 및 변환 적용
image = Image.open("../datasets/images/cat.jpg")
transformed_image = transform(image)

print(f"변환된 이미지 shape: {transformed_image.shape}")  # (C, H, W)
plt.imshow(transformed_image.permute(1, 2, 0))  # (H, W, C)로 변환하여 표시
plt.title("Resize + ToTensor")
plt.show()

---

#### 예제 4.18 회전 및 뒤집기 (Rotation, Flip)

In [ ]:
from PIL import Image
from torchvision import transforms


transform = transforms.Compose(
    [
        # RandomRotation: 무작위 회전 (-30° ~ +30°)
        # expand=False: 이미지 크기 유지 (잘림 발생 가능)
        transforms.RandomRotation(degrees=30, expand=False, center=None),
        # RandomHorizontalFlip: 50% 확률로 좌우 뒤집기
        transforms.RandomHorizontalFlip(p=0.5),
        # RandomVerticalFlip: 50% 확률로 상하 뒤집기
        transforms.RandomVerticalFlip(p=0.5)
    ]
)

image = Image.open("../datasets/images/cat.jpg")
transformed_image = transform(image)
plt.imshow(transformed_image)
plt.title("Rotation + Flip")
plt.show()

---

#### 예제 4.19 자르기 및 패딩 (Crop, Pad)

In [ ]:
from PIL import Image
from torchvision import transforms


transform = transforms.Compose(
    [
        # RandomCrop: 무작위 위치에서 512x512 크기로 자르기
        transforms.RandomCrop(size=(512, 512)),
        # Pad: 이미지 주변에 50픽셀 패딩 추가
        # fill: 패딩 색상 (R, G, B)
        # padding_mode: "constant", "edge", "reflect", "symmetric"
        transforms.Pad(padding=50, fill=(127, 127, 255), padding_mode="constant")
    ]
)

image = Image.open("../datasets/images/cat.jpg")
transformed_image = transform(image)
plt.imshow(transformed_image)
plt.title("Crop + Pad")
plt.show()

---

#### 예제 4.20 크기 조정 (Resize)

In [ ]:
from PIL import Image
from torchvision import transforms


transform = transforms.Compose(
    [
        # Resize: 이미지 크기를 512x512로 조정
        # 비율 유지 안 함 (왜곡 발생 가능)
        transforms.Resize(size=(512, 512))
    ]
)

image = Image.open("../datasets/images/cat.jpg")
transformed_image = transform(image)
plt.imshow(transformed_image)
plt.title("Resize (512x512)")
plt.show()

---

#### 예제 4.21 아핀 변환 (Affine Transformation)

In [ ]:
from PIL import Image
from torchvision import transforms


transform = transforms.Compose(
    [
        # RandomAffine: 무작위 아핀 변환
        # degrees: 회전 범위 (-15° ~ +15°)
        # translate: 이동 범위 (가로/세로 각각 20%)
        # scale: 스케일 범위 (0.8배 ~ 1.2배)
        # shear: 기울기 범위 (-15° ~ +15°)
        transforms.RandomAffine(
            degrees=15, translate=(0.2, 0.2),
            scale=(0.8, 1.2), shear=15
        )
    ]
)

image = Image.open("../datasets/images/cat.jpg")
transformed_image = transform(image)
plt.imshow(transformed_image)
plt.title("Affine Transformation")
plt.show()

---

#### 예제 4.22 색상 변환 및 정규화 (ColorJitter, Normalize)

In [ ]:
from PIL import Image
from torchvision import transforms


transform = transforms.Compose(
    [
        # ColorJitter: 색상 변환
        # brightness: 밝기 변화 범위 (0.7 ~ 1.3)
        # contrast: 대비 변화 범위
        # saturation: 채도 변화 범위
        # hue: 색조 변화 범위 (-0.3 ~ 0.3)
        transforms.ColorJitter(
            brightness=0.3, contrast=0.3,
            saturation=0.3, hue=0.3
        ),
        transforms.ToTensor(),
        # Normalize: ImageNet 평균/표준편차로 정규화
        # 사전학습 모델 사용 시 필수
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        transforms.ToPILImage()  # 시각화를 위해 다시 PIL로 변환
    ]
)

image = Image.open("../datasets/images/cat.jpg")
transformed_image = transform(image)
plt.imshow(transformed_image)
plt.title("ColorJitter + Normalize")
plt.show()

---

#### 예제 4.23 imgaug을 활용한 고급 증강 (노이즈, 날씨 효과)

In [ ]:
import numpy as np
np.bool = np.bool_  # Deprecated 오류 방지
from PIL import Image
from torchvision import transforms
from imgaug import augmenters as iaa


class IaaTransforms:
    """imgaug 증강을 torchvision과 함께 사용하기 위한 래퍼 클래스"""
    
    def __init__(self):
        # Sequential: 여러 증강을 순차적으로 적용
        self.seq = iaa.Sequential([
            # SaltAndPepper: 소금-후추 노이즈 (3%~7%)
            iaa.SaltAndPepper(p=(0.03, 0.07)),
            # Rain: 비 효과 (속도 0.3~0.7)
            iaa.Rain(speed=(0.3, 0.7))
        ])
    
    def __call__(self, images): 
        images = np.array(images)
        print(f"입력 이미지: {images.shape}, dtype: {images.dtype}")
        augmented = self.seq.augment_image(images)
        return Image.fromarray(augmented)


transform = transforms.Compose([
    IaaTransforms()
])

image = Image.open("../datasets/images/cat.jpg")
transformed_image = transform(image)
plt.imshow(transformed_image)
plt.title("SaltAndPepper + Rain Effect")
plt.show()

---

#### 랜덤 삭제 (Random Erasing)

이미지의 일부 영역을 무작위로 삭제/대체

In [ ]:
from PIL import Image
from torchvision import transforms


transform = transforms.Compose([
    transforms.ToTensor(),
    # RandomErasing: 무작위 영역 삭제
    # p=1.0: 100% 확률로 적용
    # value=0: 검은색으로 대체
    transforms.RandomErasing(p=1.0, value=0),
    # value='random': 랜덤 색상으로 대체
    transforms.RandomErasing(p=1.0, value='random'),
    transforms.ToPILImage()
])

image = Image.open("../datasets/images/cat.jpg")
transformed_image = transform(image)
plt.imshow(transformed_image)
plt.title("Random Erasing")
plt.show()

---

#### 예제 4.24 Mixup

두 이미지를 가중 평균으로 혼합

In [ ]:
import numpy as np
from PIL import Image
from torchvision import transforms


class Mixup:
    """Mixup 증강: 두 이미지를 가중 평균으로 혼합"""
    
    def __init__(self, target, scale, alpha=0.5, beta=0.5):
        """
        Args:
            target: 혼합할 대상 이미지
            scale: 대상 이미지 크기 조정
            alpha: 원본 이미지 가중치
            beta: 대상 이미지 가중치
        """
        self.target = target
        self.scale = scale
        self.alpha = alpha
        self.beta = beta

    def __call__(self, image):
        image = np.array(image)
        target = self.target.resize(self.scale)
        target = np.array(target)
        # Mixup: image * alpha + target * beta
        mix_image = image * self.alpha + target * self.beta
        return Image.fromarray(mix_image.astype(np.uint8))


transform = transforms.Compose(
    [
        transforms.Resize((512, 512)),
        Mixup(
            target=Image.open("../datasets/images/dog.jpg"),
            scale=(512, 512),
            alpha=0.5,  # 고양이 이미지 50%
            beta=0.5    # 강아지 이미지 50%
        )
    ]
)

image = Image.open("../datasets/images/cat.jpg")
transformed_image = transform(image)
plt.imshow(transformed_image)
plt.title("Mixup (Cat 50% + Dog 50%)")
plt.show()